La descripció teòrica dells recents experiments sobre la condensació Bose-Einstein (BEC) es basen en l'equació de Gross-Pitaevskii (GP). Els àtoms estan confinats dins un camp magnètic, els efectes del qual estan ben descrits per un potencial harmònic. 
Aquí tractem un condensat de $^{87}Rb$. Assumim que tots els àtoms són al condensat, $\psi(\vec{r})$ normalitzada tal que:
$$\int \lvert\psi(\vec{r})\rvert^2 \, d \vec{r} = N$$
En unitats d'oscil·lador harmònic ($\hbar\omega$), l'equació GP és:
$$\left[-\frac{1}{2} \nabla^2 + \frac{1}{2} r_1^2 + 4\pi\bar{a}_s N\lvert\bar{\psi}(\vec{r}_1)\rvert^2\right] \bar{\psi}(\vec{r}_1) = \bar{\mu}\bar{\psi}(\vec{r}_1)$$
on $\mu$ és el potencial químic, $N$ el nombre de partícules i $\bar{a}_s$ la longitud de dispersió d'ona s. En aquest cas, $\bar{\psi}(\vec{r}_1)$ està normalitzada a 1.

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

#----------------------------------------------------------
# Definim les funcions necessàries per efectuar els càlculs
#----------------------------------------------------------

def segona_der(f, df, h, n): # Calcula la segona derivada fent servir diferències finites, pas h, n iteracions
    df[0] = 0.0
    # Punts interiors
    for i in range(1, n-1):
        df[i] = (f[i-1] + f[i+1] - 2.0*f[i]) / (h**2.0)
    # A l'extrem:
    df[n] = (f[n-1] - 2.0*f[n]) / (h**2.0)
    return

#----------------------------------------------------------
# EXPLICACIÓ DE LES VARIABLES:
# xr : coordenada radial
# xmu : Potencial químic local
# step : pas d'integració a l'espai r
# npunts : Número de punts a la xarxa radial
# psi : Funció d'ona actual
# psider : Segona derivada de la funció d'ona
# psiant : Funció d'ona a la iteració anterior
# den : Densitat de partícules
# u : Energia potencial total d'una
# ene0 : Energia total
# radius : Radi quadràtic mig
# radius2 : Radi al quadrat
# ekin : Energia cinètica
# poth0 : Energia potencial harmònica
# potself : Eenergia potencial d'interacció
# chem : Potencial químic promig
# xaver : Valor promig addicional
# normden : Normalització de la densitat
# as3n : Paràmetre de densitat
#----------------------------------------------------------
# Arxiu d'inputs
inputs = np.loadtxt('grosspita.grau.dat', dtype=np.float64)
print(inputs)

psi = np.zeros([npunts, 1])
xr = np.zeros([npunts, 1])
# Configurem els paràmetres d'interacció
cequ = a0 * npart  # si =0 es recupera el resultat de l'OH
u = np.zeros([npunts, 1])
den = np.zeros([npunts, 1])


def grosspita(alpha, npunts, step, a0, npart, timestep, niter, psi, xr, cequ):
    radius = 0.0
    radius2 = 0.0
    ekin = 0.0
    poth0 = 0.0
    potself = 0.0  
    xaver = 0.0
    normden = 0.0
    as3n = npart * (a0**3.0)  
    psinova = np.zeros([npunts, 1])
    psiant = np.zeros([npunts, 1])
    psider = np.ones([npunts, 1])
    xmu = np.ones([npunts, 1])
    energia = np.zeros([npunts, 1])
    ene0 = 0.0
    chem = 0.0
    #----------------------------------------------------------
    # Inicialitzem la nostra funció d'ona inicial (HO)
    #----------------------------------------------------------
    alpha2 = alpha**2.0
    cvar = 2.0 * np.sqrt(alpha)**3.0 / np.sqrt(np.sqrt(np.pi))
    xnorm = 0.0
    for i in range(npunts):
        if i != 0 :
            xr[i] = step * (i-1)*1.0
        xr2 = xr[i]**2.0
        psiant[i] = cvar * xr[i] * np.exp(-0.5 * alpha2 * xr2)
        psi[i] = psiant[i]


    for it in tqdm(range(niter)):
        segona_der(psi, psider, step, int(npunts)-1)   # Calculem el Laplacià    
        for i in range(npunts): # Càlcul d'energia i potencial químic local
            xr2 = xr[i]**2.0
            if (psi[i] == 0.0):
                xmu[i] = 0.0
                energia[i] = 0.0
            else:
                # Contribucions a l'energia
                #energia[i] = energia[i-1] - 0.5 * (psi[i]*psider[i] + xr2*(psi[i]**2.0) + cequ*xr2*((psi[i]/xr[i])**4.0))
                # Potencial químic:
                xmu[i] = -0.5*psider[i]/psi[i] + 0.5*xr2 + cequ*(psi[i]/xr[i])**2.0
            # Evolució temporal de la funció d'ona
            psinova[i] = psi[i] - timestep*xmu[i]*psi[i]
            # Acumulació per a la normalització
            xnorm = xnorm + (psinova[i]**2.0)
        
        # Normalització i actualització
        xnorm = np.sqrt(xnorm * step)
        #energia[i] = energia[i] * step
        psi[i] = psinova[i] / xnorm
    # Calculem les integrals necessàries
    for i in range(1, npunts):
        xr2 = xr[i]**2.0        
        if ((xr[i] != 0.0) and (psi[i] != 0.0)):
            radius = radius + xr2 * (psi[i]**2.0)
            ekin = ekin + psi[i] * psider[i]
            poth0 = poth0 + xr2 * (psi[i]**2.0)
            potself = potself + xr2 * (psi[i] / xr[i])**4.0
            chem = chem + xmu[i] * (psi[i]**2.0)
            # Potencial i densitat monoparticular:
            u[i] = 0.5 * xr2 + cequ * (psi[i] / xr[i])**2.0
            den[i] = (psi[i] / xr[i])**2.0
        # Normalitzacions:
        normden = normden + den[i] * xr2
        xaver = xaver + (psi[i]**2.0) * as3n * den[i]

    # Normalitzem els resultats:
    radius2 = radius * step
    radius = np.sqrt(radius * step)
    xaver = xaver * step
    chem = chem * step 
    ekin = -0.5 * ekin * step 
    poth0 = 0.5 * poth0 * step 
    potself = 0.5 * potself * step * cequ
    ene0 = ekin + poth0 + potself 
    print('energia:', ene0, 'chem:', chem)
    return

[[4.33e-03 7.00e+02 1.50e-02 1.00e+06 5.00e-05 3.00e-01 7.00e+04]
 [4.33e-03 4.00e+02 1.50e-02 1.00e+04 1.00e-04 8.00e-01 4.00e+04]
 [4.33e-03 6.00e+02 1.50e-02 1.00e+05 1.00e-04 4.00e-01 5.00e+04]
 [4.33e-03 4.00e+02 2.00e-02 1.00e+03 1.00e-04 5.00e-01 5.00e+04]
 [4.33e-03 3.00e+02 2.00e-02 1.00e+02 1.00e-04 5.00e-01 5.00e+04]]


$a_0$) Fent servir el programa GP, comprovar que posant la interacció a 0 es recuperen els resultats per a l'oscil·lador harmònic (HO). Per què l'energia per partícula és igual al potencial químic?

In [ ]:
# Fem servir els valors de la segona dels paràmetres d'entrada
alpha = inputs[1,5]
npunts = int(inputs[1,1])
step = inputs[1,2]
a0 = inputs[1,0]    # Longitud de dispersió en unitats de l'oscil·lador harnònic
npart = inputs[1,3] # Número de partícules
timestep = inputs[1,4] # Pas temporal
niter = int(inputs[1,6]) # Número d'iteracions
grosspita(alpha, npunts, step, a0, npart, timestep, 2000, psi, xr, 0.0 )

100%|██████████| 2000/2000 [00:24<00:00, 82.77it/s]


energia: [1.65186002] chem: [1.65186002]


: 

En aquest cas, en anunlar el terme d'interacció, el sistema recupera la interacció HO i l'equació GP queda:
$$\left[-\frac{1}{2} \nabla^2 + \frac{1}{2} r_1^2 \right] \bar{\psi}(\vec{r}_1) = \bar{\mu}\bar{\psi}(\vec{r}_1)$$
Per tant, el potencial químic es l'autoenergia de l'aestat en aquest cas.

Teòricament, el resultats per a l'energia i el potencial químic en una trampa harmònica 3d és $\epsilon = \frac{3}{2}\hbar\omega$. En el nostre càlcul, fet amb 10000 nuclis i amb 40000 iteracions, obtenim (en unitats d'HO):
$$\epsilon = \bar{\mu} = 1.65 \approx \frac{3}{2}$$
Que és un resultat prou proper al teòric.